In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
openai = OpenAI()

In [5]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad you're here. How can I assist you today?


In [6]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [12]:
# Let's try one out. Change the website and add print statements to follow along.

#ed = Website("https://www.superdatascience.com/")
ed = Website("https://www.theblogstarter.com/")
print(ed.title)
print(ed.text)

How to Start a Blog in 2025 - Easy Guide to Create a Blog for Beginners
Home
Blog Tutorial
About Me
Contact Me
Skip to content
Home
Blog Tutorial
About Me
Contact Me
How to Start A Blog in 2025
Create a Blog in 20 Minutes
Updated January 1st, 2025
Are you looking for an easy guide on
how to start a blog
?
The step-by-step guide on this page will show you how to create a blog in 20 minutes with
just the most basic computer skills.
After completing this guide you will have a beautiful blog that is ready to share with the world.
This guide is made especially for beginners.  I will walk you through each and every step, using plenty of pictures and videos to make it all perfectly clear.
If you get stuck or have questions at any point, simply
send me a message
and I will do my best to help you out.
Ready to start?
Click here to skip the intro and start building your blog now
My name is Scott Chow, and I am going to show you how to start blogging today. I have been building blogs and websites

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [13]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [14]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [16]:
user_prompt_for(ed)

'You are looking at a website titled How to Start a Blog in 2025 - Easy Guide to Create a Blog for Beginners\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nBlog Tutorial\nAbout Me\nContact Me\nSkip to content\nHome\nBlog Tutorial\nAbout Me\nContact Me\nHow to Start A Blog in 2025\nCreate a Blog in 20 Minutes\nUpdated January 1st, 2025\nAre you looking for an easy guide on\nhow to start a blog\n?\nThe step-by-step guide on this page will show you how to create a blog in 20 minutes with\njust the most basic computer skills.\nAfter completing this guide you will have a beautiful blog that is ready to share with the world.\nThis guide is made especially for beginners.  I will walk you through each and every step, using plenty of pictures and videos to make it all perfectly clear.\nIf you get stuck or have questions at any point, simply\nsend me a message\nand I

In [17]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [18]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled How to Start a Blog in 2025 - Easy Guide to Create a Blog for Beginners\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nBlog Tutorial\nAbout Me\nContact Me\nSkip to content\nHome\nBlog Tutorial\nAbout Me\nContact Me\nHow to Start A Blog in 2025\nCreate a Blog in 20 Minutes\nUpdated January 1st, 2025\nAre you looking for an easy guide on\nhow to start a blog\n?\nThe step-by-step guide on this page will show you how to create a blog in 20 minutes with\njust the most basic computer skills.\nAfter completing this guide you will have a beautiful blog that is ready to share with the world.\nThis guide is

In [19]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [20]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner Website\n\nEdward Donner\'s website features his interests and professional background, primarily focused on coding and experimenting with Large Language Models (LLMs). He is the co-founder and CTO of Nebula.io, a company that utilizes AI to enhance talent discovery and management. Previously, he founded the AI startup untapt, which was acquired in 2021.\n\n## Key Sections:\n- **About**: Ed shares his passion for technology, DJing, and electronic music production, along with his professional journey in the AI industry.\n- **Outsmart**: This section highlights a project called "Outsmart," an arena designed to compare LLMs in strategic scenarios involving diplomacy and cunning.\n\n## Recent Announcements:\n- **December 21, 2024**: Welcome message for SuperDataScientists.\n- **November 13, 2024**: Resource sharing for mastering AI and LLM engineering.\n- **October 16, 2024**: Information on transitioning from software engineering to AI data science.\n- **August

In [21]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [24]:
display_summary("https://www.theblogstarter.com/")

# How to Start a Blog in 2025 - Summary

The website titled "How to Start a Blog in 2025" is a comprehensive guide designed for beginners looking to create their own blogs. Authored by Scott Chow, who has extensive blogging experience since 2002, it aims to simplify the process of starting a blog in just 20 minutes, even for those with basic computer skills.

## Key Features of the Guide:
- **Step-by-step Instructions:** The guide breaks down the blogging process into manageable steps, including:
  1. Choosing a blog name and topic.
  2. Getting the blog online with recommended hosting (BlueHost).
  3. Customizing the blog using WordPress.
  4. Writing and publishing the first blog post.
  5. Promoting the blog to attract readers.
  6. Monetizing the blog through various strategies.

- **Beginner-Friendly:** The tutorial includes visuals and videos to facilitate understanding, and the author encourages users to reach out for personalized help if needed.

- **Passion Over Expertise:** It reassures readers that they don't need to be expert writers; rather, a passion for their chosen topic is essential to connect with an audience.

- **Potential Benefits of Blogging:**
  - Sharing personal stories and experiences.
  - Earning passive income through various monetization methods.
  - Gaining recognition in niche areas and finding community engagement.

- **Updates and New Year’s Sale:** The guide was recently updated on January 1, 2025, and notes a special promotion for BlueHost at a reduced rate of $1.99 per month for those who sign up through their links.

Overall, the website serves as an all-in-one resource for anyone interested in starting a blog, with practical advice and support available from the author.